In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import src.NearNear_v3 as nn
%load_ext autoreload
%autoreload 2

In [2]:
#y = df.pop('sale_amount')
#X = df
#X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

#### Test `compute_distances` function used in Model
 - Use toy dataset with items in SF and ATX (roughly)
 - Pop off a 2 rows in Austin as a 'test case'
 - Expect distance matrix to include 
     - zeros for ATX-ATX and SF-SF 
     - ~1475mi for SF-ATX cases

In [13]:
df = pd.read_csv('data/test_data.csv')
df

,property_latitude,property_longitude,property_imprating,sale_amount
0,38,-122,A,10
1,38,-122,A,10
2,38,-122,A,10
3,38,-122,B,20
4,38,-122,B,20
5,38,-122,B,20
6,30,-98,C,30
7,30,-98,C,30
8,30,-98,C,30


In [14]:
predictors = ['property_latitude','property_longitude','property_imprating']
X_train = df.loc[:7,predictors]
X_test = df.loc[7:,predictors]
y_train = df.loc[:7,'sale_amount']
y_test = df.loc[7:,'sale_amount']

In [15]:
lat_train = X_train['property_latitude'].values
lon_train = X_train['property_longitude'].values
lat_test = X_test['property_latitude'].values
lon_test = X_test['property_longitude'].values
nn.compute_distances(lat_train,lon_train,lat_test,lon_test)

array([[1477.12735462, 1477.12735462, 1477.12735462, 1477.12735462,
        1477.12735462, 1477.12735462,    0.        ,    0.        ],
       [1477.12735462, 1477.12735462, 1477.12735462, 1477.12735462,
        1477.12735462, 1477.12735462,    0.        ,    0.        ]])

#### Test - model uses LatLong to establish comparables
 - Use toy dataset with items in SF and ATX (roughly)
 - Pop off a single row in Austin as a 'test case'
 - Expect model returns average value of remaining items in ATX (30)

In [3]:
df = pd.read_csv('data/test_data.csv')
df

,property_latitude,property_longitude,property_imprating,sale_amount
0,38,-122,A,10
1,38,-122,A,10
2,38,-122,A,10
3,38,-122,B,20
4,38,-122,B,20
5,38,-122,B,20
6,30,-98,C,30
7,30,-98,C,30
8,30,-98,C,30


In [4]:
predictors = ['property_latitude','property_longitude','property_imprating']
X_train = df.loc[:8,predictors]
X_test = df.loc[8:,predictors]
y_train = df.loc[:8,'sale_amount']
y_test = df.loc[8:,'sale_amount']

In [5]:
nn_model = nn.NearNear(lat='property_latitude',
                        lon='property_longitude')
nn_model.fit(X_train,y_train)
y_pred = nn_model.predict(X_test)

In [6]:
print('Predicted value for our test case is: {}'.format(y_pred[0]))

Predicted value for our test case is: 30.0


#### Test - model uses LatLong and `cat_comp` parameter to establish comparables
 - Use toy dataset with items in SF and ATX (roughly)
